In [9]:
import librosa
import json
import math
import statistics
import librosa.display
import scipy as sp
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns 
from sklearn.cluster import DBSCAN, KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from tkinter import Tk     # from tkinter import Tk for Python 3.x
from tkinter.filedialog import askopenfilename

#OPEN FILE
Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
filename = askopenfilename() # show an "Open" dialog box and return the path to the selected file
file_dir = filename
print(file_dir)
#LOAD SONG
signal, samplerate = librosa.load(file_dir)
#GET GENERAL TEMPO
signal_bpm, samplerate_bpm = librosa.load(file_dir, duration = 60, sr = 10000)
onset_env = librosa.onset.onset_strength(signal_bpm, sr=samplerate_bpm)
tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=samplerate_bpm)
#DIVIDE INTO COMPASES
song_duration= librosa.get_duration(y=signal, sr=samplerate) #duration of the song
beat_num = 1
bpm = tempo
vcompas_time = beat_num * 60 / bpm #duration of a compas                                       
vcompas_num  = song_duration / vcompas_time                  #number of compases 
compas_sample_num = vcompas_time * samplerate                #number of samples on each compas
song_sample_num = compas_sample_num * vcompas_num            #total number of samples in the song
vcompas_data = {} 
for vcompas_i in range(0,int(vcompas_num)):                                
    start = math.floor(compas_sample_num * vcompas_i)          #start sample for the compas
    end = math.floor(compas_sample_num * (vcompas_i + 1))      #end sample for the compas
    selected_samples = signal[start:end]                       #array with all the compas samples
    vcompas_data[vcompas_i] = ([selected_samples],vcompas_time * vcompas_i)
df_compas = pd.DataFrame.from_dict(vcompas_data, orient = 'index', columns = ['wform','times'])
#signals
df_means = df_compas
df_means['max'] = df_means.wform.apply(lambda x: x[0].max())

max_signal = max(list(df_means['max']))
min_signal = min(list(df_means['max']))
signal_umbral = max_signal - min_signal

max_noise = 0.8
min_noise = 0.35
noise_umbral = max_noise - min_noise

df_means['gnre'] = df_means['max'].apply(lambda x: (x * noise_umbral / signal_umbral) + min_noise)

df_noises = df_means[['times','gnre']]
df_noises['times'] = df_noises['times'].apply(lambda x: x[0])

df_means = df_compas
df_means['max'] = df_means.wform.apply(lambda x: x[0].max())

max_signal = max(list(df_means['max']))
min_signal = min(list(df_means['max']))
signal_umbral = max_signal - min_signal

max_noise = 1.2
min_noise = 0
noise_umbral = max_noise - min_noise

df_means['gnre'] = df_means['max'].apply(lambda x: round(((x * noise_umbral / signal_umbral) + min_noise), 4))

df_noises_hip_rot = df_means[['times','gnre']]
df_noises_hip_rot['times'] = df_noises['times'].apply(lambda x: x)


#CREATE A DATA FRAME WITH THE FREQUENCIES ARRAY
df_freq = pd.DataFrame(columns = ['mag']) 
for csignal in df_compas['wform']:
    csignal = csignal[0]
    X = np.fft.fft(csignal)
    X_mag = np.absolute(X)
    #print(X_mag[0])
    newdf = pd.DataFrame([[X_mag]], columns = ['mag'])
    df_freq = pd.concat([df_freq, newdf], axis = 0)

df_freq = df_freq.reset_index(drop = True)
#GET MAXIMUS OF THE FREQUENCIES
instrument_division = [0,100,300,600,1000,2500,7000,15000]
fixed_instrument_div = list(instrument_division)
fixed_instrument_div.remove(0)

f = np.linspace(0, samplerate, len(X_mag), endpoint = False)
df_bin_freq = pd.DataFrame(columns = fixed_instrument_div)

for ampl in df_freq.mag: #loop time domain compases
    
    freq_full = [] #list with al freq list

    for i in range(1,len(instrument_division)): #loop through freq domain
        
        start = instrument_division[i - 1] * len(ampl) / f[len(ampl) - 2] #starting point of the frequency bin
        end = instrument_division[i] * len(ampl) / f[len(ampl) - 2] #end point
        
        freq_bin = ampl[math.floor(start):math.floor(end)] #save magintudes bin
        freq_full += [freq_bin]
        #print(freq_bin)
        #dat_tuple = (list(ampl).index(ampl[start:end].max()),signl[start:end].max())
        #print(f[dat_tuple[0]], dat_tuple)

    newdf_cluster_freq = pd.DataFrame([freq_full], columns = fixed_instrument_div)
    df_bin_freq = pd.concat([df_bin_freq, newdf_cluster_freq])
    
df_bin_freq = df_bin_freq.reset_index()
df_bin_freq.drop(['index'], inplace = True, axis = 1)
#LOWER
df_lower = df_bin_freq.drop(columns = [600,1000,2500,7000,15000])
df_lower.columns = list(map(lambda x: str(x), df_lower.columns))
df_lower['100_mean'] = df_lower['100'].apply(lambda x: x.mean())
df_lower['100_median'] = df_lower['100'].apply(lambda x: statistics.median(x))
df_lower['100_max'] = df_lower['100'].apply(lambda x: x.max())
df_lower['300_mean'] = df_lower['300'].apply(lambda x: x.mean())
df_lower['300_median'] = df_lower['300'].apply(lambda x: statistics.median(x))
df_lower['300_max'] = df_lower['300'].apply(lambda x: x.max())
df_lower['noise'] = df_means['gnre']
df_lower_model = df_lower.drop(['100','300'], axis = 1)
scaler = StandardScaler().fit(df_lower_model)
X_prep = scaler.transform(df_lower_model)
kmeans = GaussianMixture(n_components= 2, covariance_type='diag',max_iter =  100,n_init = 8, random_state = 1234)
kmeans.fit(X_prep)
yhat = kmeans.predict(X_prep)
clusters = np.unique(yhat)
df_low_final = df_compas.drop(columns = ['wform'])
df_low_final['times'] = df_low_final['times'].apply(lambda x: x[0])
df_low_final['gnre'] = yhat
#MID
df_mid = df_bin_freq.drop(columns = [100,300,2500,7000,15000])
df_mid.columns = list(map(lambda x: str(x), df_mid.columns))
df_mid['600_mean'] = df_mid['600'].apply(lambda x: x.mean())
df_mid['600_median'] = df_mid['600'].apply(lambda x: statistics.median(x))
df_mid['600_max'] = df_mid['600'].apply(lambda x: x.max())
df_mid['1000_mean'] = df_mid['1000'].apply(lambda x: x.mean())
df_mid['1000_median'] = df_mid['1000'].apply(lambda x: statistics.median(x))
df_mid['1000_max'] = df_mid['1000'].apply(lambda x: x.max())
df_mid['noise'] = df_means['gnre']
df_mid_model = df_mid.drop(['600','1000'], axis = 1)
scaler = StandardScaler().fit(df_mid_model)
X_prep = scaler.transform(df_mid_model)
kmeans = GaussianMixture(n_components= 2, covariance_type='spherical',max_iter = 300,n_init = 8, random_state = 1234, init_params = 'random')
kmeans.fit(X_prep)
yhat = kmeans.predict(X_prep)
clusters = np.unique(yhat)
df_mid_final = df_compas.drop(columns = ['wform'])
df_mid_final['times'] = df_mid_final['times'].apply(lambda x: x[0])
df_mid_final['gnre'] = yhat
#HIGH
df_high = df_bin_freq.drop(columns = [100,300,600,1000])
df_high.columns = list(map(lambda x: str(x), df_high.columns))
df_high['2500_mean'] = df_high['2500'].apply(lambda x: x.mean())
df_high['2500_median'] = df_high['2500'].apply(lambda x: statistics.median(x))
df_high['2500_max'] = df_high['2500'].apply(lambda x: x.max())
df_high['7000_mean'] = df_high['7000'].apply(lambda x: x.mean())
df_high['7000_median'] = df_high['7000'].apply(lambda x: statistics.median(x))
df_high['7000_max'] = df_high['7000'].apply(lambda x: x.max())
df_high['15000_mean'] = df_high['15000'].apply(lambda x: x.mean())
df_high['15000_median'] = df_high['15000'].apply(lambda x: statistics.median(x))
df_high['15000_max'] = df_high['15000'].apply(lambda x: x.max())
df_high['noise'] = df_means['gnre']
df_high_model = df_high.drop(['2500','7000','15000'], axis = 1)
scaler = StandardScaler().fit(df_mid_model)
X_prep = scaler.transform(df_mid_model)
kmeans = GaussianMixture(n_components= 4, covariance_type='spherical',max_iter = 100,n_init = 20, random_state = 1234, init_params = 'kmeans')
kmeans.fit(X_prep)
yhat = kmeans.predict(X_prep)
clusters = np.unique(yhat)
df_high_final = df_compas.drop(columns = ['wform'])
df_high_final['times'] = df_high_final['times'].apply(lambda x: x[0])
df_high_final['gnre'] = yhat
#FULL
full_freq = pd.concat([df_high_model, df_mid_model, df_lower_model], axis = 1)
scaler = StandardScaler().fit(full_freq)
X_prep = scaler.transform(full_freq)
kmeans = KMeans(n_clusters= 3,
               init="random",
               n_init= 2,  # try with 1, 4, 8, 20, 30, 100...
               max_iter= 300,
               tol= 1e-4,
               algorithm="auto",
               random_state=1234)
kmeans.fit(X_prep)
yhat = kmeans.predict(X_prep)
clusters = np.unique(yhat)
df_full_final = df_compas.drop(columns = ['wform'])
df_full_final['times'] = df_full_final['times'].apply(lambda x: x[0])
df_full_final['gnre'] = yhat
#FEATURES
step_freq = 1/(vcompas_time * 2)
noise_freq = 1/(vcompas_time * 2)
#SAVING
dict_full = {}
dict_low = {}
dict_mid = {}
dict_high = {}
dict_legs = {}
dict_noises = {}
dict_noises_hand = {} 

for i in range(0,len(df_low_final.times)):
    dict_low[str(df_low_final.times[i])] = str(df_low_final.gnre[i])
    dict_mid[str(df_mid_final.times[i])] = str(df_mid_final.gnre[i])
    dict_high[str(df_high_final.times[i])] = str(df_high_final.gnre[i])
    dict_legs[str(df_full_final.times[i])] = str(df_full_final.gnre[i])
    dict_noises[str(df_noises.times[i])] = str(df_noises.gnre[i])
    dict_noises_hand[str(df_noises_hip_rot.times[i])] = str(df_noises_hip_rot.gnre[i])
    
dict_full['low'] = str(dict_low)
dict_full['mid'] = str(dict_mid)
dict_full['high'] = str(dict_high)
dict_full['legs'] = str(dict_legs)
dict_full['noises'] = str(dict_noises)
dict_full['hnoises'] = str(dict_noises_hand)
dict_full['freqs'] = str(step_freq[0])
json_dump = json.dumps(dict_full)
with open('../json/poses.json', 'w') as f:
    f.write(json_dump)
with open(r'C:\Users\urgpa\OneDrive\Escritorio\Unity\PuppetTest\Assets\_Scripts\Poses\poses.json', 'w') as f:
    f.write(json_dump)
    
print('SONG PROCESSED AND SAVED')

C:/Users/urgpa/OneDrive/Escritorio/Unity/PuppetTest/Assets/_Sonidos/jammin.wav


C:\Users\urgpa\anaconda3\lib\site-packages\librosa\filters.py:238: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn(
<ipython-input-9-113b7f0357f1>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_noises['times'] = df_noises['times'].apply(lambda x: x[0])
<ipython-input-9-113b7f0357f1>:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_noises_hip_rot['times'] = df_noi

SONG PROCESSED AND SAVED
